In [1]:
import numpy as np
from scipy.sparse import csr_matrix

In [2]:
def definition(inputFile):
    hashMap = {}
    with open(inputFile) as f:
        content = f.read().splitlines()

    for i in xrange(len(content)):
        (index, definition) = content[i].split('\t')
        hashMap[index] = definition
        
    return (hashMap, list(sorted(hashMap.keys())))

In [3]:
(authors, aIndices) = definition('author.txt')
(papers, pIndices) = definition('paper.txt')
(terms, tIndices) = definition('term.txt')
(venues, vIndices) = definition('venue.txt')

In [4]:
def formMatrices(inputFile):
    # need to create M_AP, M_PV, M_PT
    ap_row = []
    ap_col = []
    pv_row = []
    pv_col = []
    pt_row = []
    pt_col = []

    with open(inputFile) as f:
        content = f.read().splitlines()

    for i in xrange(len(content)):
        (paper, item, _) = content[i].split('\t')

        if (int(item) <= 13575): # this is a term => update M_PT
            pt_row.append(pIndices.index(paper))
            pt_col.append(tIndices.index(item))
        elif (42145 <= int(item) and int(item) <= 42164): # venue => update M_PV
            pv_row.append(pIndices.index(paper))
            pv_col.append(vIndices.index(item))
        else: # author => update M_AP
            ap_row.append(aIndices.index(item))
            ap_col.append(pIndices.index(paper))

    AP = csr_matrix((np.ones(len(ap_row)), (ap_row, ap_col)), dtype=np.float64)
    PT = csr_matrix((np.ones(len(pt_row)), (pt_row, pt_col)), dtype=np.float64)
    PV = csr_matrix((np.ones(len(pv_row)), (pv_row, pv_col)), dtype=np.float64)
    
    return (AP*np.transpose(AP),
            AP*PT*np.transpose(PT)*np.transpose(AP),
            AP*PV*np.transpose(PV)*np.transpose(AP))

In [5]:
(APA, APTPA, APVPA) = formMatrices('relation.txt')

In [6]:
def write2file(name, matrix):
    with open(name + '.txt', 'w') as f:
        (row, col) = matrix.nonzero()
        for i in xrange(len(row)):
            f.write(row[i].__str__() + '\t' + col[i].__str__() + '\t' + matrix[row[i], col[i]].__str__() + '\n')

In [ ]:
write2file('APTPA', APTPA)

In [ ]:
write2file('APVPA', APVPA)

In [7]:
APTPA

<5000x5000 sparse matrix of type '<type 'numpy.float64'>'
	with 24690038 stored elements in Compressed Sparse Row format>

In [8]:
APVPA

<5000x5000 sparse matrix of type '<type 'numpy.float64'>'
	with 11243494 stored elements in Compressed Sparse Row format>